In [5]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [6]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord_u = 2
ord_phi = ord_u - 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*(G/KBTV)* Trace(F.trans*F ) + H


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord_u, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord_u, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord_phi) # fes phi

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((1,1,1))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
phi = CF(-z)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        #LHS
        
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx)
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx 
        if ord_phi == 0:
            BF+= -delta * exp(psih) * vphi * dx -eps * (delta * vphi * dx)
        else:
            BF+= -delta * exp(psih) * vphi * dx -eps * (grad(delta) * grad(vphi) * dx)
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    






In [7]:
""" WORKING PARAMETERS FOR SIMPLEST BONDED TEST"""
### 1 PHI = -Z
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 25

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*50 ,softening_n))

### 2

In [8]:
SetNumThreads(8)
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 30

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*30 ,softening_n))


for num, load in enumerate(gammas):
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it = 20
        tol_newton = 1e-6
        tol_QN = 1e-8
        tol_delta = 1e-8
    for k in range(1,max_PG_it):

        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        if k != 0:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later
        print("Starting QN")
        for i in range(max_iter_qnewton):
            G_funs = u,v,alpha,delta,psih,psik 
            with TaskManager():
                BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            
            
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
            #iterations of newton for the NL problem at each QN step
            print("Solving nonlinear subproblem")
            for iter in range(max_iter_newton):
                with TaskManager():
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                    gfu.vec.data += -w
                if abs(InnerProduct(w,res)) < tol_newton:
                    print("Tol achieved", InnerProduct(w,res))
                    break
            # check tolerance on QN
            qn_tol_val= Integrate((gfu.components[1])**2,mesh)
            if qn_tol_val < tol_delta:
                print("QN tol achieved",qn_tol_val )
                break
            # update psih
            psih.vec.data = psih.vec + gfu.components[1].vec
        # check error on gfu
        print("Error",Integrate((uk-gfu.components[0])**2,mesh))

    Draw(gfu.components[0],mesh)



PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved -8.389400163452473e-07
Solving nonlinear subproblem
Tol achieved -4.111057020778921e-07
Solving nonlinear subproblem
Tol achieved -2.68407084978225e-07
Solving nonlinear subproblem
Tol achieved -5.916490129813282e-07
Solving nonlinear subproblem
Tol achieved -4.3091031738909334e-07
QN tol achieved 3.9043748962896454e-08
Error 34.95896058137446
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 5.861655589005923e-07
Solving nonlinear subproblem
Tol achieved -5.56597767289724e-07
Solving nonlinear subproblem
Tol achieved -6.136055917262492e-07
Solving nonlinear subproblem
Tol achieved -8.06063572049003e-07
QN tol achieved 4.931112540488044e-08
Error 13.547260789918063
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 4.990180015967061e-07
Solving nonlinear subproblem
Tol achieved -2.6248456504456817e-07
Solving nonlinear subproblem
Tol achieved -6.536740512127875e-07
Solving nonlinear subproblem
Tol a

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 2.9754225411396667e-07
Solving nonlinear subproblem
Tol achieved -2.790295963317606e-07
QN tol achieved 1.4141066232253349e-08
Error 0.0064509000741113705
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 6.647436731104967e-07
Solving nonlinear subproblem
Tol achieved -4.64901883476314e-07
QN tol achieved 5.182458864084736e-08
Error 0.0011937092915274697
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 2.5916555690621973e-07
Solving nonlinear subproblem
Tol achieved -5.784974817582766e-07
QN tol achieved 2.7127880371668285e-08
Error 0.0012967894427479167
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 8.901820942012039e-07
Solving nonlinear subproblem
Tol achieved -3.3964649238844866e-07
QN tol achieved 3.058095246578009e-08
Error 0.0007179173933529016
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 3.211530442617325e-07
Solving nonlinear subproblem
Tol achieved -3.02

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 4.707686734713378e-07
Solving nonlinear subproblem
Tol achieved -4.402509651521958e-07
QN tol achieved 1.440364167595816e-08
Error 0.007105752092661647
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 2.605639106393046e-07
Solving nonlinear subproblem
Tol achieved -7.279756537772476e-07
QN tol achieved 2.8818712543630506e-08
Error 0.0012848488910842785
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 4.064596099755677e-07
Solving nonlinear subproblem
Tol achieved -9.086844152185002e-07
QN tol achieved 2.8931340723706497e-08
Error 0.0014295046773533935
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.526321974973981e-07
Solving nonlinear subproblem
Tol achieved -5.440886918144144e-07
QN tol achieved 1.648589933657387e-08
Error 0.0008177993194117486
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 5.210087588068471e-07
Solving nonlinear subproblem
Tol achieved -4.96136

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 8.907992156907437e-07
Solving nonlinear subproblem
Tol achieved -7.437399315161116e-07
QN tol achieved 4.648871836835868e-09
Error 0.021813171422006634
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 4.399388474067551e-07
Solving nonlinear subproblem
Tol achieved -3.207629392484365e-07
QN tol achieved 2.9897562713361555e-09
Error 0.002560710736781571
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 7.941389315046838e-07
Solving nonlinear subproblem
Tol achieved -4.956575464840895e-07
QN tol achieved 9.498521302250938e-10
Error 0.003528246471666773
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 9.06523573208775e-07
Solving nonlinear subproblem
Tol achieved -4.1188099890396613e-07
QN tol achieved 3.789290098737903e-10
Error 0.0028742746447663423
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 4.882382758339116e-07
Solving nonlinear subproblem
Tol achieved -5.74900686

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 2.61727239583238e-07
Solving nonlinear subproblem
Tol achieved -8.628756106576068e-07
QN tol achieved 3.5002881920083314e-09
Error 0.025558200470714845
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 5.10374217823456e-07
Solving nonlinear subproblem
Tol achieved -3.7410131179594636e-07
QN tol achieved 3.2279569553320674e-09
Error 0.0028138421874063933
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 9.392321218870689e-07
Solving nonlinear subproblem
Tol achieved -5.952039332687488e-07
QN tol achieved 9.952719939944459e-10
Error 0.003973081476464573
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 2.7837136184683605e-07
Solving nonlinear subproblem
Tol achieved -5.177306066252037e-07
QN tol achieved 1.406530936634659e-09
Error 0.0033861515148393413
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 6.321336812920143e-07
Solving nonlinear subproblem
Tol achieved -7.659336

KeyboardInterrupt: 

In [9]:
# DO VTK    
vtk = VTKOutput(ma=mesh,coefs=[gfu.components[0],gfu.components[1]],names=["u","phi"],filename="gel")

vtk.Do()

'gel'